# core

> Fill in a module description here

In [91]:
#| default_exp core

In [92]:
#| hide
from nbdev.showdoc import *

In [93]:
#| export
from vsdx import VisioFile
import rdflib
import os
from pprint import pprint
import requests
from vsdx_rdf.visualize import VisualizeClient

In [94]:
#| export
class Client:

    @staticmethod
    def main(path, output_dir, page=-1, verbose=False):

        client = Client(path, verbose=verbose)

        client.createNodesAndEdges(page)
        client.convertToRdf()

        paths = client.save(output_dir)

        for path in paths:
                
            if os.path.exists(path) == False:
                continue
            
            output_path = path.replace(".ttl", ".png")

            with open(path, "r") as f:

                text = f.read()

            VisualizeClient.graph_draw_by_kanzaki(text, output_path)



    def __init__(self, path, verbose=False):
        self.vis = VisioFile(path)

        self.verbose = verbose

    def createNodesAndEdges(self, page):

        vis = self.vis

        resultsByPage = {}

        # pages = [vis.pages[1]]

        if page == -1:

            pages = vis.pages

        else:
                
            pages = [vis.pages[page]]

        for page, i  in zip(pages, range(len(pages))):

            # print(page)

            title = page.name

            ####

            connects = page.connects

            edges = {}

            for connect in connects:

                edge_id = connect.from_id

                if edge_id not in edges:

                    edges[edge_id] = {}

                # from_id = connect.from_id

                from_id = None
                to_id = None

                if connect.from_rel == "BeginX":

                    from_id = connect.to_id

                    edges[edge_id]["from"] = from_id


                else:

                    to_id = connect.to_id

                    edges[edge_id]["to"] = to_id


            #### 

            children = page.child_shapes

            nodes = {}

            for child in children:

                # print(child)

                child_id = child.ID
                child_name = child.text

                # print(child.text)

                shape_name = child.shape_name
                master_page_ID = child.master_page_ID

                # print(master_page_ID)

                # shage_type = child.shape_type

                

                if child_id in edges:
                        
                    edges[child_id]["name"] = child_name.strip()

                else:

                    '''
                    if self.verbose:
                        print(f"child_id: {child_id}, child_name: {child_name}, shape_name: {shape_name}, shage_type: {shage_type}")
                    '''

                    if shape_name is None:

                        # print(child.geometry)

                        if master_page_ID == "7":

                            shape_name = "Rectangle"

                        elif "Ellipse" in str(child.geometry):
                            shape_name = "Circle"
                        
                        elif "RelLineTo" in str(child.geometry):
                            shape_name = "Rectangle"
                        
                        else:
                            # shape_name = "Rectangle"
                            shape_name = "Circle"
                            pass

                        # continue
                        # shape_name = "Circle" # 要検討

                    nodes[child_id] = {
                        "name": child_name.strip()
                    }

                    if "Circle" in shape_name or "Ellipse" in shape_name:

                        nodes[child_id]["type"] = "resource"

                    else:

                        nodes[child_id]["type"] = "literal"


            resultsByPage[i] = {
                "title": title,
                "nodes": nodes,
                "edges": edges
            }

        
        if self.verbose:
            print("----- resultsByPage -----")
            pprint(resultsByPage)
            print("-------------------------")
        # ''''''

        self.resulsByPage = resultsByPage

    def convertToRdf(self):

        prefixes = {
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
        }

        resultsByPage = self.resulsByPage

        for page in resultsByPage:

            # title = resultsByPage[page]["title"]

            nodes = resultsByPage[page]["nodes"]

            edges = resultsByPage[page]["edges"]

            # RDFグラフを初期化
            g = rdflib.Graph()

            for prefix in prefixes:
                g.bind(prefix, rdflib.Namespace(prefixes[prefix]))

            '''
            # 名前空間の定義
            namespace = rdflib.Namespace("http://example.org/node/")

            g.bind("node", namespace)

            ns_property = rdflib.Namespace("http://example.org/edge/")

            g.bind("edge", ns_property)
            '''

            namespace = rdflib.Namespace("http://example.org/")
            g.bind("ex", namespace)

            uris = {}

            literals = {}

            # ノード（リソース）をグラフに追加
            for node_id, info in nodes.items():

                if info["type"] == "resource":

                    name = info["name"] # .replace("\n", "_")

                    print("name", name)

                    if name.startswith("http"):
                        node_uri = name

                    elif ":" in name:
                        
                        for prefix in prefixes:
                                
                            if prefix in name:
    
                                name = name.replace(prefix + ":", prefixes[prefix])

                                break

                        node_uri = name

                    else:

                        name = name.replace("\n", "_")

                        # print(info["name"])

                        node_uri = namespace[name]

                    if node_id not in uris:
                            
                        uris[node_id] = node_uri

                else:
                        
                    literals[node_id] = info["name"]

            ###

            # リレーションシップをグラフに追加
            for rel_id, rel_info in edges.items():
                # from_id = namespace[rel_info['from']]
                # to_id = namespace[rel_info['to']]

                

                from_id = rel_info['from']

                if "to" not in rel_info:
                    print("to not found")
                    continue

                to_id = rel_info['to']

                if from_id not in uris:
                    continue

                from_uri = uris[from_id]

                # property_uri = rdflib.URIRef(f"http://schema.org/type") if rel_info['name'] == "タイプ" else rdflib.URIRef(f"http://example.org/title")

                name = rel_info['name'] or "base"

                if name.startswith("http"):
                    property_uri = rdflib.URIRef(name)
                elif ":" in name:
                    for prefix in prefixes:
                        if prefix in name:
                            name = name.replace(prefix + ":", prefixes[prefix])
                            break
                    property_uri = rdflib.URIRef(name)

                else:
                    name = name.replace("\n", "_")
                    property_uri = rdflib.URIRef(namespace[name])

                if to_id in literals:
                    g.add((rdflib.URIRef(from_uri), property_uri, rdflib.Literal(literals[to_id])))

                else:
                    if to_id not in uris:
                        continue

                    to_uri = uris[to_id]

                    g.add((rdflib.URIRef(from_uri), property_uri, rdflib.URIRef(to_uri)))


            # RDFを出力
            # print(g.serialize(format='turtle'))

            self.resulsByPage[page]["rdf"] = g # g.serialize(format='turtle').decode()

    def save(self, output_dir):

        resultsByPage = self.resulsByPage

        paths = []

        for page in resultsByPage:

            title = resultsByPage[page]["title"]

            g = resultsByPage[page]["rdf"]

            output_path = f"{output_dir}/{title}.ttl"

            paths.append(output_path)

            text = g.serialize(format='turtle')

            if text.strip() == "":
                continue

            os.makedirs(output_dir, exist_ok=True)

            with open(output_path, "w") as f:

                f.write(g.serialize(format='turtle'))

        # print("done")

        return paths


In [95]:
#| hide
import nbdev; nbdev.nbdev_export()